# Урок 7. Сверточные нейронные сети для анализа текста

## Задание
Берем отызывы за лето (из архива с материалами или предыдущего занятия)

1. Обучить conv сеть для классификации. Рассмотреть 2-а варианта сеточек<br>
    1.1 Инициализировать tf.keras.layers.Embedding предобученными векторами взять к примеру с https://rusvectores.org/ru/<br>
    1.2 Инициализировать слой tf.keras.layers.Embedding по умолчанию (ну то есть вам ничего не делать с весами)<br>

Сравнить две архитектуры с предобученными весами и когда tf.keras.layers.Embedding обучается сразу со всей сеточкой, что получилось лучше.

## Полезные ссылки
 - [Использование предобученных эмбедингов1](https://blog.keras.io/using-pre-trained-word-embeddings-in-a-keras-model.html)
 - [Использование предобученных эмбедингов2](https://keras.io/examples/nlp/pretrained_word_embeddings/)
 - [предобученные модели word2vec fasttext (смотреть Статические модели)](https://rusvectores.org/ru/models/)

## Решение.

### Подготовка данных.

In [1]:
# !pip install xlrd
# !pip install stop_words

# import numpy as np
# !pip install gensim
# !pip install numpy==1.20.0
# !pip install pycocotools==2.0.0

In [2]:
import os
import pandas as pd

In [3]:
df = pd.read_excel('отзывы за лето.xls')

In [4]:
df = df[['Rating','Content']]
df.head()

,Rating,Content
0,5,It just works!
1,4,В целом удобноное приложение...из минусов хотя...
2,5,Отлично все
3,5,Стал зависать на 1% работы антивируса. Дальше ...
4,5,"Очень удобно, работает быстро."


In [5]:
df['Rating'].value_counts()

5    14586
1     2276
4     2138
3      911
2      748
Name: Rating, dtype: int64

Установи порог отзывов 3 бала. если значение выше порога отзыв положительный. Если ниже- отрицательный.

In [6]:
df['Rating'] = df['Rating'].apply(lambda x: 1 if x > 3 else 0)

In [7]:
df['Rating'].value_counts()

1    16724
0     3935
Name: Rating, dtype: int64

### Предобработка текста.

Т.к. в работе мы будем использовать предобученную модель **rusvectores**, то предобработку текста выполним в используя пример с этого сайта.

In [8]:
max_words = 1000
max_sentence_len = 100

In [9]:
from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re
from sklearn.model_selection import train_test_split
from utils import apostrophe_dict, emoticon_dict, short_word_dict  # см. файл utils.py
from  rus_preprocessing_udpipe import process

import sys
import os
import wget
import re
from ufal.udpipe import Model, Pipeline
from tqdm import tqdm

tqdm.pandas()

/home/sergey/anaconda3/envs/deeppavlov_env/lib/python3.7/site-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [10]:
df.head(5)

,Rating,Content
0,1,It just works!
1,1,В целом удобноное приложение...из минусов хотя...
2,1,Отлично все
3,1,Стал зависать на 1% работы антивируса. Дальше ...
4,1,"Очень удобно, работает быстро."


In [11]:
# Теперь повторим это для всех записей.
def replase_words(text,dict_=emoticon_dict): 
    text = str(text)
    output = ''
    for word in text.split(' '): # не будем делить текст на части будем искать подстроку в строке. это касается только смайликов.
        word = word.strip()
        if word in dict_.keys(): 
            output += ' ' + dict_[word]
        else:
            output += ' ' + word
    return output

In [12]:
# txt = replase_words(txt, emoticon_dict)
df['Content'] = df['Content'].apply(replase_words)

Загрузка предобученной модели для NER.

In [13]:
udpipe_model_url = "https://rusvectores.org/static/models/udpipe_syntagrus.model"
udpipe_filename = udpipe_model_url.split("/")[-1]

if not os.path.isfile(udpipe_filename):
    print("UDPipe model not found. Downloading...", file=sys.stderr)
    wget.download(udpipe_model_url)

# print("\nLoading the model...", file=sys.stderr)
model = Model.load(udpipe_filename)
process_pipeline = Pipeline(
    model, "tokenize", Pipeline.DEFAULT, Pipeline.DEFAULT, "conllu"
)

Проверка

In [14]:
# print("Processing input...", file=sys.stderr)
# for input_line in sys.stdin:
#     break
res = 'очень странный текст о котором мы хотим поговорить'#unify_sym(input_line.strip())
print(res)
output = process(process_pipeline, text=res)
print(" ".join(output))

очень странный текст о котором мы хотим поговорить
очень_ADV странный_ADJ текст_NOUN о_ADP который_PRON мы_PRON хотеть_VERB говорить_VERB


In [15]:
# Попробуем тоже самое сделать и для нашего датасета.
df['Content'] = df['Content'].progress_apply(lambda x:" ".join(process(process_pipeline, text=x)))

100%|██████████| 20659/20659 [02:38<00:00, 129.95it/s]


Раделим DataFrame на ренировочную и тестовую выборки !!! 

In [16]:
df_train, df_test = train_test_split(df,test_size=0.3)

In [17]:
train_corpus = " ".join(df_train["Content"])

In [18]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download("punkt")

tokens = word_tokenize(train_corpus)

[nltk_data] Downloading package punkt to /home/sergey/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [19]:
tokens_filtered = [word for word in tokens]# if word.isalnum()]
tokens_filtered

['очень_ADV',
 'удобный_ADJ',
 'оригинальный_ADJ',
 'прошивка_NOUN',
 'определять_VERB',
 'как_SCONJ',
 'неоригинать_NOUN',
 'из_ADP',
 'за_ADP',
 'что_PRON',
 'ограничивать_VERB',
 'функциональность_NOUN',
 'приложение_NOUN',
 'отличный_ADJ',
 'понятный_ADJ',
 'удобный_ADJ',
 'быстро_ADV',
 'реально_ADV',
 'классный_ADJ',
 'приложение_NOUN',
 'рекомендовать_VERB',
 'подключаться_VERB',
 'крутяк_NOUN',
 'ставить_VERB',
 'тройка_NOUN',
 'за_ADP',
 'то',
 ',',
 'что2_NOUN',
 'изз_NOUN',
 'встроенный_ADJ',
 'антивирус_NOUN',
 'или_CCONJ',
 'мочь_VERB',
 'по_ADP',
 'какой_DET',
 'то_SCONJ',
 'другой_ADJ',
 'причина_NOUN',
 'вышибать_VERB',
 'приложение_NOUN',
 'на_ADP',
 'ввод_NOUN',
 'пин_NOUN',
 'код_NOUN',
 'после_ADP',
 'вход_NOUN',
 'в_ADP',
 'л/к.очедывать_NOUN',
 'напрягает.не_ADV',
 'уверенный_ADJ',
 'в_ADP',
 'безопасность_NOUN',
 'приложение_NOUN',
 '.проша_VERB',
 'исправлять_VERB',
 'весь_PRON',
 'отличный_ADJ',
 'спасибо_NOUN',
 'не_PART',
 'очень_ADV',
 'удобный_ADJ',
 'отлич

Отфильтруем данные и соберём в корпус N наиболее частых токенов

In [20]:
from nltk.probability import FreqDist
dist = FreqDist(tokens_filtered)
tokens_filtered_top = [pair[0] for pair in dist.most_common(max_words-1)]

In [21]:
tokens_filtered_top[:10]

['приложение_NOUN',
 'не_PART',
 'удобный_ADJ',
 'и_CCONJ',
 'очень_ADV',
 'все_PRON',
 'в_ADP',
 'я_PRON',
 'на_ADP',
 'работать_VERB']

In [22]:
vocabulary = {v: k for k, v in dict(enumerate(tokens_filtered_top, 1)).items()}

In [23]:
vocabulary

{'приложение_NOUN': 1,
 'не_PART': 2,
 'удобный_ADJ': 3,
 'и_CCONJ': 4,
 'очень_ADV': 5,
 'все_PRON': 6,
 'в_ADP': 7,
 'я_PRON': 8,
 'на_ADP': 9,
 'работать_VERB': 10,
 'с_ADP': 11,
 'отличный_ADJ': 12,
 'хороший_ADJ': 13,
 ',': 14,
 'нравиться_VERB': 15,
 'раз_NOUN': 16,
 'но_CCONJ': 17,
 'по_ADP': 18,
 'что_SCONJ': 19,
 'телефон_NOUN': 20,
 'после_ADP': 21,
 'мочь_VERB': 22,
 'а_CCONJ': 23,
 'x_NUM': 24,
 'это_PRON': 25,
 'спасибо_PROPN': 26,
 'что_PRON': 27,
 'обновление_NOUN': 28,
 'весь_DET': 29,
 'при_ADP': 30,
 'так_ADV': 31,
 'супер_NOUN': 32,
 'за_ADP': 33,
 'пользоваться_VERB': 34,
 'и_PART': 35,
 'только_PART': 36,
 'быть_AUX': 37,
 'банк_NOUN': 38,
 'быть_VERB': 39,
 'делать_VERB': 40,
 'нет_VERB': 41,
 'весь_PRON': 42,
 'антивирус_NOUN': 43,
 'пароль_NOUN': 44,
 'вход_NOUN': 45,
 'заходить_VERB': 46,
 'быстро_ADV': 47,
 'вы_PRON': 48,
 'устраивать_VERB': 49,
 'у_ADP': 50,
 'карта_NOUN': 51,
 'как_SCONJ': 52,
 'прошивка_NOUN': 53,
 'он_PRON': 54,
 'для_ADP': 55,
 'этот_DET'

In [24]:
import numpy as np

def text_to_sequence(text, maxlen):
    result = []
    tokens = word_tokenize(text.lower())
    tokens_filtered = [word for word in tokens ] #if word.isalnum()]
    for word in tokens_filtered:
        if word in vocabulary:
            result.append(vocabulary[word])
    padding = [0]*(maxlen-len(result))
    return padding + result[-maxlen:]

In [25]:
x_train = np.asarray([text_to_sequence(text, max_sentence_len) for text in df_train["Content"]], dtype=np.int32)
x_test = np.asarray([text_to_sequence(text, max_sentence_len) for text in df_test["Content"]], dtype=np.int32)

# Обучим нейронную сеть.

In [26]:
import numpy as np
import tensorflow.keras
import tensorflow as tf

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D,GlobalMaxPooling1D

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import TensorBoard 
from tensorflow.keras.callbacks import EarlyStopping  
from sklearn.metrics import accuracy_score 

In [27]:
num_classes = 2
epochs = 20
batch_size = 512
print_batch_n = 100

 __1.1__ Инициализировать tf.keras.layers.Embedding предобученными векторами взять к примеру с https://rusvectores.org/ru/<br>

Загрузим архим с предобученным Embedding-ом.

In [28]:
# !wget "http://vectors.nlpl.eu/repository/20/180.zip"
# !unzip "180.zip"

Поскольку обучение и загрузка моделей могут занимать продолжительное время, иногда бывает полезно вести лог событий. Для этого используется стандартная питоновская библиотека logging.

In [29]:
import sys
import gensim
import logging

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [30]:
word_model = gensim.models.KeyedVectors.load_word2vec_format('180/model.bin', binary=True)

2021-09-09 15:35:26,130 : INFO : loading projection weights from 180/model.bin
2021-09-09 15:35:33,486 : INFO : loaded (189193, 300) matrix from 180/model.bin


In [31]:
pretrained_weights = word_model.vectors
vocab_size, emdedding_size = pretrained_weights.shape
print(f'vocab_size: {vocab_size}, emdedding_size: {emdedding_size}')

vocab_size: 189193, emdedding_size: 300


In [32]:
train_count = len(df_train)
test_count = len(df_test)

In [33]:
df_train.head()

,Rating,Content
16675,1,очень_ADV удобный_ADJ
5333,0,оригинальный_ADJ прошивка_NOUN определять_VERB...
11400,1,отличный_ADJ
17316,1,понятный_ADJ удобный_ADJ быстро_ADV реально_AD...
18935,1,крутяк_NOUN


In [34]:
def word2idx(word):
  return word_model.key_to_index[word] # если нет такого токена. то возвращаем пустую строку. или вообще не добавляем элемент.
def idx2word(idx):
  return word_model.index_to_key[idx]


train_y = tensorflow.keras.utils.to_categorical(df_train["Rating"], num_classes)
train_x = np.zeros([train_count, max_sentence_len], dtype=np.int32)

for i, sentence in enumerate(df_train['Content']):
  tokens = word_tokenize(sentence)
  for t, word in enumerate(tokens):
    try:
      train_x[i, t] = word2idx(word) # отбрасывам все токены, которые отсутствуют в предобученной модели.
    except:
      continue

print('train_x shape:', train_x.shape)
print('train_y shape:', train_y.shape)

train_x shape: (14461, 100)
train_y shape: (14461, 2)


In [35]:
x = df_test['Content']
y = df_test['Rating']

test_y = tensorflow.keras.utils.to_categorical(df_test["Rating"], num_classes)
test_x = np.zeros([test_count, max_sentence_len], dtype=np.int32)

for i, sentence in enumerate(df_test['Content']):
  tokens = word_tokenize(sentence)
  for t, word in enumerate(tokens):
    try:
      test_x[i, t] = word2idx(word)
    except:
      continue

print('test_x shape:', test_x.shape)
print('test_y shape:', test_y.shape)

test_x shape: (6198, 100)
test_y shape: (6198, 2)


In [36]:
train_y[:5]

array([[0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.]], dtype=float32)

In [37]:
train_x[0]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [38]:
word_model.vectors[0]

array([ 1.0967804e+00, -2.2944486e+00,  1.9791678e+00,  3.4804371e-01,
        4.0753922e-01,  1.3286122e+00, -9.3368673e-01,  5.4947221e-01,
       -6.8077618e-01, -7.4963701e-01, -8.0936104e-02,  6.5788299e-02,
       -8.8564938e-01,  5.0793958e-01, -1.0864110e+00, -4.3317631e-01,
        2.0482888e-02, -5.7119979e-03, -1.0036458e+00,  3.1729680e-01,
        1.1956499e+00,  1.0685917e+00, -8.9309484e-01,  7.4419886e-01,
        4.4171312e-01, -2.0080043e-01, -2.6623638e+00,  1.7598321e-01,
       -2.0019765e+00, -5.6796205e-01, -2.0350738e-01, -7.3997623e-01,
        8.0226004e-01,  1.4174094e+00,  1.0990121e-01,  1.2313192e+00,
        1.6655500e+00, -2.6187131e-01,  1.7604357e+00, -3.1903556e-01,
        2.4266930e+00, -1.2458172e+00,  1.0440445e+00,  2.7978971e+00,
       -8.2489556e-01,  4.3603179e-01,  9.2311478e-01, -1.8360819e-01,
       -6.3582733e-02,  2.9669294e-01,  5.6388801e-01,  2.5535300e+00,
        2.3046949e+00, -1.8354168e+00, -6.8992484e-01, -1.3858901e+00,
      

In [39]:
# Создадим слой Embedding.
embedidding = Embedding(input_dim=vocab_size, output_dim=emdedding_size, weights=[pretrained_weights], trainable=False)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


2021-09-09 15:35:37,847 : WARNING : From /home/sergey/anaconda3/envs/deeppavlov_env/lib/python3.7/site-packages/tensorflow_core/python/keras/initializers.py:119: calling RandomUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [40]:
model = Sequential()
model.add(embedidding)
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


2021-09-09 15:35:38,223 : WARNING : From /home/sergey/anaconda3/envs/deeppavlov_env/lib/python3.7/site-packages/tensorflow_core/python/ops/resource_variable_ops.py:1630: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [41]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [42]:
tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss')  


history = model.fit(train_x, train_y,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Train on 13014 samples, validate on 1447 samples
Epoch 1/20
13014/13014 [==============================] - 9s 718us/sample - loss: 0.5415 - acc: 0.7840 - val_loss: 0.4943 - val_acc: 0.8058
Epoch 2/20
13014/13014 [==============================] - 11s 871us/sample - loss: 0.4904 - acc: 0.8074 - val_loss: 0.4925 - val_acc: 0.8058
Epoch 3/20
13014/13014 [==============================] - 12s 901us/sample - loss: 0.4907 - acc: 0.8074 - val_loss: 0.4925 - val_acc: 0.8058


In [43]:
results = model.predict(test_x, batch_size=batch_size, verbose=1)

6198/6198 [==============================] - 1s 236us/sample


In [44]:
pred_y = np.argmax(results,axis = 1)

In [45]:
rusvectores_accuracy = accuracy_score(np.argmax(test_y,axis = 1), pred_y)
rusvectores_accuracy

0.8149403033236527

__1.2__ Инициализировать слой tf.keras.layers.Embedding по умолчанию (ну то есть вам ничего не делать с весами)<br>

In [46]:
y_train = tensorflow.keras.utils.to_categorical(df_train["Rating"], num_classes)
y_test = tensorflow.keras.utils.to_categorical(df_test["Rating"], num_classes)

In [47]:
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_sentence_len))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPooling1D())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [48]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 128)          128000    
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 98, 128)           49280     
_________________________________________________________________
activation_3 (Activation)    (None, 98, 128)           0         
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1290      
_________________________________________________________________
activation_4 (Activation)    (None, 10)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 2)                

In [49]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [50]:
tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss')  


history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Train on 13014 samples, validate on 1447 samples
Epoch 1/20
13014/13014 [==============================] - 7s 501us/sample - loss: 0.5263 - acc: 0.8073 - val_loss: 0.4908 - val_acc: 0.8058
Epoch 2/20
13014/13014 [==============================] - 9s 696us/sample - loss: 0.4875 - acc: 0.8074 - val_loss: 0.4869 - val_acc: 0.8086
Epoch 3/20
13014/13014 [==============================] - 8s 599us/sample - loss: 0.4840 - acc: 0.8084 - val_loss: 0.4838 - val_acc: 0.8093
Epoch 4/20
13014/13014 [==============================] - 8s 595us/sample - loss: 0.4833 - acc: 0.8087 - val_loss: 0.4821 - val_acc: 0.8079
Epoch 5/20
13014/13014 [==============================] - 8s 590us/sample - loss: 0.4808 - acc: 0.8103 - val_loss: 0.4831 - val_acc: 0.8100


In [51]:
results = model.predict(x_test, batch_size=batch_size, verbose=1)

6198/6198 [==============================] - 1s 108us/sample


In [52]:
y_pred = np.argmax(results,axis = 1)

In [53]:
simple_accuracy = accuracy_score(df_test['Rating'].to_numpy(), y_pred)
simple_accuracy

0.8175217812197483

In [54]:
print('Результат:')
print(f'1. rusvectores: {rusvectores_accuracy:.4f}, 2. simple: {simple_accuracy:.4f}  ')

Результат:
1. rusvectores: 0.8149, 2. simple: 0.8175  


## Вывод

В ранних тестах у меня модель с предобученным эмбедингом показывала результат несколько лучше чем модель на эмбединг небыл предобучен. Но, сейчас модели показываю одинаковый результат, с той разницей что на модели с предобученным эмбеддингом модель обучается немоного быстрее.


TODO: Предобработка текста в соответствии с рекомендациями rusvectores показала результат хуже чем ту что мы делали на уроках. см. файл HW_NLP_Lesson_7_1. Когда будет время. необходимо будет переделать предобработку.